In [11]:
#runs pre trained model and stores in output in a nc file. 
# Basics
import os
import sys
import numpy as np
#os.environ["PROJ_LIB"] = "/global/homes/a/anayak/.conda/envs/pytorch-gpu/share/proj"
os.environ["PROJ_LIB"] = "/glade/p/cgd/amp/jet/collections/anaconda3/envs/lbltorch/share/proj"

sys.path.append('/glade/u/home/jet/ClimateNet/ncar')

# Netcdf4
import datetime
from netCDF4 import Dataset

#architecture
from architecture import CGNet_wrap as cg
from architecture import deeplab_xception

# Torch
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader

from dataset import cam_dataset_nc as nc_dataset
#from utils import visualizeUtil,utils
from utils import utils

In [12]:

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

if torch.cuda.is_available():
   print("cuda is available: using gpu")
   map_location=lambda storage, loc: storage.cuda()
else:
   print("cuda is not available: using cpu")
   map_location='cpu'
#device = torch.device("cpu")
#map_location='cpu'
#use_cuda = False

cuda is not available: using cpu


In [13]:
arch = "cgnet" # "cgnet"

In [14]:
#data and output paths
data_dir = "/glade/scratch/jet/ClimateNet/Climate_data/" 
if arch == "deep" :
    model_path = "/glade/u/home/jet/ClimateNet/ncar/architecture/ClimateNet_model.pth"
    output_dir = "/glade/scratch/jet/ClimateNet/output.deep/"
    net = deeplab_xception.DeepLabv3_plus(nInputChannels=4, n_classes=3, os=16)
else:
    model_path = "/glade/u/home/jet/ClimateNet/ncar/architecture/CGNet_wrap.pth" #change to your desired pth file 
    output_dir = "/glade/scratch/jet/ClimateNet/output.cgnet/"
    net = cg.CGNet(classes=3, channels=4, M=3, N=21, dropout_flag=True)

    #if torch.cuda.device_count() > 1:
print("Let's use", torch.cuda.device_count(), device)
# dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
net = torch.nn.parallel.DataParallel(net)

have droput layer
Let's use 0 cpu


In [15]:


checkpoint = torch.load(model_path, map_location=map_location)
#checkpoint = torch.load(model_path,map_location=device) #gpu or cpu
#print(checkpoint.keys())
#print(checkpoint['net_state_dict'])
#checkpoint = torch.load(model_path,map_location='cpu') #cpu
net.load_state_dict(checkpoint['net_state_dict'])
net.eval()


DataParallel(
  (module): CGNet(
    (level1_0): ConvBNPReLU(
      (padding): Wrap()
      (conv): Conv2d(4, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=32)
    )
    (level1_1): ConvBNPReLU(
      (padding): Wrap()
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=32)
    )
    (level1_2): ConvBNPReLU(
      (padding): Wrap()
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=32)
    )
    (sample1): InputInjection(
      (pool): ModuleList(
        (0): Wrap()
        (1): AvgPool2d(kernel_size=3, stride=2, padding=0)
      )
    )
    (sample2): InputInj

In [16]:
#gpu
#device = torch.device("cuda")
net.to(device)

DataParallel(
  (module): CGNet(
    (level1_0): ConvBNPReLU(
      (padding): Wrap()
      (conv): Conv2d(4, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=32)
    )
    (level1_1): ConvBNPReLU(
      (padding): Wrap()
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=32)
    )
    (level1_2): ConvBNPReLU(
      (padding): Wrap()
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=32)
    )
    (sample1): InputInjection(
      (pool): ModuleList(
        (0): Wrap()
        (1): AvgPool2d(kernel_size=3, stride=2, padding=0)
      )
    )
    (sample2): InputInj

In [17]:
import importlib
importlib.reload(nc_dataset)
#from dataset import cam_dataset as nc_dataset
#get data
#key names as in nc
#var_list = ["TCWV","U","V","MSL","longitude","latitude"] #Order matters: #TMQ, U850, V850, PSL, Longitude, Latitude
var_list = ["TMQ","U850","V850","PSL","lon","lat","date","datesec"] #Order matters: #TMQ, U850, V850, PSL, Longitude, Latitudedataset = nc_dataset.NetcdfDataset(data_dir,variables=var_list[0:4]) #optional: subset=true, condition='regexpconditon'
dataset = nc_dataset.NetcdfDataset(data_dir,variables=var_list[0:4]) #optional: subset=true, condition='regexpconditon'
print(dataset.ds.variables["time"][:])

<class 'list'>
['/glade/scratch/jet/ClimateNet/Climate_data/b.e13.B20TRC5CN.ne120_g16.003.deeplab.2000.poissonfill.nc']
Initialized dataset with max length of 2920  time slices
Initialized dataset with dataPath= /glade/scratch/jet/ClimateNet/Climate_data/
Initialized dataset with file= ['/glade/scratch/jet/ClimateNet/Climate_data/b.e13.B20TRC5CN.ne120_g16.003.deeplab.2000.poissonfill.nc']
[54750.125 54750.25  54750.375 ... 55114.75  55114.875 55115.   ]


In [18]:
print(var_list[0:4])
print(var_list[4])
print(dataset.variables[0])
#variables_array = np.array([dataset.variables[var] for var in var_list])
x,y=dataset.__getitem__(0)
#print(x)
#print(y)




['TMQ', 'U850', 'V850', 'PSL']
lon
TMQ


In [19]:
print(x.shape)
print(x[0].shape)
print(y)
#print(x[0][3][3][0][0])

(4, 768, 1152)
(768, 1152)
54750.125


In [20]:
loader = DataLoader(dataset, batch_size=10)

In [21]:
for i, (inputs, time) in enumerate(loader):
    print("batch # ",i," time:",time[0])
    #input tensor size should be 1 X 4 X W X H
    inputs=inputs.squeeze(0)    
    inputs=utils.normalize_0_1_2_7(inputs) #normalize data 
    print(device)
    #push data on GPU and pass forward
    inputs = Variable(inputs, requires_grad=True).to(device)
    with torch.no_grad():
        outputs = net(inputs)
    predictions = torch.max(outputs, 1)[1].cpu().numpy();
    outfile = str(dataset.case)+str("_")+str(time[0].numpy())+str(".nc")
    if outfile:
        ncout = Dataset(output_dir + outfile ,'w',clobber=True,format='NETCDF4') # using netCDF4 for output format
        ncout.createDimension('time',None)
        ncout.createDimension('lon',dataset.ds.variables['lon'].shape[0])
        ncout.createDimension('lat',dataset.ds.variables['lat'].shape[0])
        timevar = ncout.createVariable('time','float32',('time'))
        timevar[:] = time
        lonvar = ncout.createVariable('lon','float32',('lon'))
        lonvar[:] = dataset.ds.variables['lon'][:]
        latvar = ncout.createVariable('lat','float32',('lat'))
        latvar[:] = dataset.ds.variables['lat'][:]

        myvar = ncout.createVariable('output_tag','u1',('time','lat','lon'));
        myvar[:] = predictions

        #ARs only
        #myvar[:] = (predictions==2).astype(int) #0-BG,1-TC,2-AR 

        #ARTMIP formatting of attributes
        timevar.setncatts({'long_name': dataset.ds.variables["time"].long_name,\
                            'units':  dataset.ds.variables["time"].units})
        lonvar.setncatts({'long_name': dataset.ds.variables["lon"].long_name,\
                            'units': dataset.ds.variables["lon"].units})
        latvar.setncatts({'long_name': dataset.ds.variables["lat"].long_name,\
                            'units': dataset.ds.variables["lat"].units})

        myvar.setncatts({'version': u"CGNet",\
                        'scheme': u"ClimateNet_DL_model",\
                        'description': u"segmentation tag"})

        ncout.close()       


batch #  0  time: tensor(54750.1250)
cpu
batch #  1  time: tensor(54751.3750)
cpu
batch #  2  time: tensor(54752.6250)
cpu
batch #  3  time: tensor(54753.8750)
cpu
batch #  4  time: tensor(54755.1250)
cpu
batch #  5  time: tensor(54756.3750)
cpu
batch #  6  time: tensor(54757.6250)
cpu
batch #  7  time: tensor(54758.8750)
cpu
batch #  8  time: tensor(54760.1250)
cpu
batch #  9  time: tensor(54761.3750)
cpu
batch #  10  time: tensor(54762.6250)
cpu
batch #  11  time: tensor(54763.8750)
cpu
batch #  12  time: tensor(54765.1250)
cpu
batch #  13  time: tensor(54766.3750)
cpu
batch #  14  time: tensor(54767.6250)
cpu
batch #  15  time: tensor(54768.8750)
cpu
batch #  16  time: tensor(54770.1250)
cpu
batch #  17  time: tensor(54771.3750)
cpu
batch #  18  time: tensor(54772.6250)
cpu
batch #  19  time: tensor(54773.8750)
cpu
batch #  20  time: tensor(54775.1250)
cpu
batch #  21  time: tensor(54776.3750)
cpu
batch #  22  time: tensor(54777.6250)
cpu
batch #  23  time: tensor(54778.8750)
cpu
ba

In [ ]:
print(device)